In [ ]:
import sys
sys.path.append("python/")

import sympy as sp
import Z1sym
import symbase
from sympy.printing.c import C99CodePrinter


# Момент $\left< Y_{l_1}^{m_1}(\phi, \theta) Y_{l_2}^{m_2*}(\phi, \theta) \right>$

In [117]:
p = Z1sym.p
px, py, pz = Z1sym.px, Z1sym.py, Z1sym.pz
Z1 = Z1sym.Z1

In [120]:
im1, im2, il1, il2 = sp.symbols("i_{m1} i_{m2} i_{l1} i_{l2}", integer=True, positive=True)
Z = sp.Symbol("Z")


def get_norm(l, m):
    norm = 1/sp.sqrt(2*sp.pi)
    norm *= sp.sqrt(sp.factorial(l-m)/sp.factorial(l+m)*(2*l+1)/2)
    if m < 0:
        m = abs(m)
        norm *= (-1)**m*sp.factorial(l-m)/sp.factorial(l+m)
    norm *= (-1)**m*2**l
    return norm


def sign(x):
    if x >= 0:
        return 1
    return -1


def binomial(n, m):
    return sp.gamma(n + 1)/(sp.gamma(m + 1)*sp.gamma(n - m + 1))


def get_moment(l1, l2, m1, m2):
    norm = get_norm(l1, m1)
    norm *= get_norm(l2, m2)
    sm1, sm2 = sign(m1), sign(m2)
    m1, m2 = abs(m1), abs(m2)

    # diff x: im1+im2
    # diff y: m1+m2-im1-im2
    # diff z: il1+il2-m1-m2

    D = sp.IndexedBase("D")
    expr = sp.Sum((sm1*sp.I)**(m1-im1)*D[im1+im2, m1+m2-im1-im2, il1+il2-m1-m2], (im1, 0, m1))
    expr = sp.Sum(expr*(-sm2*sp.I)**(m2-im2), (im2, 0, m2))
    expr = sp.Sum(expr*sp.factorial(il1)/sp.factorial(il1-m1)*binomial(l1, il1)*binomial((l1+il1-1)/2, l1), (il1, m1, l1))
    expr = sp.Sum(expr*sp.factorial(il2)/sp.factorial(il2-m2)*binomial(l2, il2)*binomial((l2+il2-1)/2, l2), (il2, m2, l2))

    expr = symbase.eval_sum_direct(expr.factor(), recursive=1)

    expr = expr.replace(sp.Indexed, lambda *args: Z1.replace(p, sp.sqrt(px**2+py**2+pz**2)).diff((px, args[1]), (py, args[2]), (pz, args[3])))
    expr = expr.subs(sp.sqrt(px**2+py**2+pz**2), p).expand()

    expr = (expr/Z1).expand()
    norm *= Z1/(sp.sinh(p)/p)*Z

    expr = expr.subs(Z1sym.subs_exp_to_m).expand()
    # expr = expr.factor()
    # expr *= norm

    lin_expr = expr.replace(Z1sym.m, p/3).factor()

    expr = C99CodePrinter().doprint(expr.subs({Z1sym.m: sp.Symbol("m")}))
    norm = C99CodePrinter().doprint(norm)
    lin_expr = C99CodePrinter().doprint(lin_expr)
    return norm, expr, lin_expr

In [126]:
lmax = 2

for l1 in range(lmax+1):
    for l2 in range(lmax+1):
        for m1 in range(-l1, l1+1):
            for m2 in range(-l2, l2+1):
                norm, expr, lin_expr = get_moment(l1, l2, m1, m2)
                print(f"if (l1 == {l1} && l2 == {l2} && m1 == {m1} && m2 == {m2})")
                print(f"return {norm} * (fabs(p) > 1e-4 ? {expr} : {lin_expr});")

if (l1 == 0 && l2 == 0 && m1 == 0 && m2 == 0)
return Z * (fabs(p) > 1e-4 ? 1 : 1);
if (l1 == 0 && l2 == 1 && m1 == 0 && m2 == -1)
return sqrt(6)*Z * (fabs(p) > 1e-4 ? (1.0/2.0)*m*p_x/p + (1.0/2.0)*I*m*p_y/p : (1.0/6.0)*(p_x + I*p_y));
if (l1 == 0 && l2 == 1 && m1 == 0 && m2 == 0)
return 2*sqrt(3)*Z * (fabs(p) > 1e-4 ? (1.0/2.0)*m*p_z/p : (1.0/6.0)*p_z);
if (l1 == 0 && l2 == 1 && m1 == 0 && m2 == 1)
return -sqrt(6)*Z * (fabs(p) > 1e-4 ? (1.0/2.0)*m*p_x/p - 1.0/2.0*I*m*p_y/p : (1.0/6.0)*(p_x - I*p_y));
if (l1 == 0 && l2 == 2 && m1 == 0 && m2 == -2)
return (1.0/3.0)*sqrt(30)*Z * (fabs(p) > 1e-4 ? -9.0/4.0*m*pow(p_x, 2)/pow(p, 3) - 9.0/4.0*I*m*p_x*p_y/pow(p, 3) + (9.0/4.0)*m*pow(p_y, 2)/pow(p, 3) + (3.0/4.0)*pow(p_x, 2)/pow(p, 2) + (3.0/4.0)*I*p_x*p_y/pow(p, 2) - 3.0/4.0*pow(p_y, 2)/pow(p, 2) : 0);
if (l1 == 0 && l2 == 2 && m1 == 0 && m2 == -1)
return (2.0/3.0)*sqrt(30)*Z * (fabs(p) > 1e-4 ? -9.0/4.0*m*p_x*p_z/pow(p, 3) - 9.0/4.0*I*m*p_y*p_z/pow(p, 3) + (3.0/4.0)*p_x*p_z/pow(p, 2) + (3.0/4